# Модуль Б.

## Загрузка библиотек

In [ ]:
# Стандартные библиотеки
import os
import shutil
import random
import time
from collections import Counter
from concurrent.futures import ThreadPoolExecutor

# Работа с данными
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image, ImageFont

# для модели yolo 
import supervision as sv
from ultralytics import YOLO

# для работы с самописной нейросетью
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
from IPython.display import Image
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

Импортированы все необходимые библиотеки.

## Разделение датасета на подвыборки

Перед началом работы с нейросетями необходимо разделить датасет на две выборки:

1. Тренировочную(предназначена для обучения)
2. Валидационную(предназначена для тестирования модели)

Деление датасета на тренировочную и валидационную выборки необходимо для предотвращения переобучения и правильной оценки качества модели. Тренировочная выборка используется для обучения модели, а валидационная — для проверки её способности обобщать на новые, неизведанные данные. Валидационная выборка помогает мониторить ошибку на данных, которые не использовались при обучении, и корректировать гиперпараметры модели, чтобы избежать переобучения.

Был выбран размер валидационной выборки в 30 процентов, поскольку у нас большой объем данных и их с запасом хватит на тренировку модели. Также важно выделить достаточно примеров для валидации, у нас в данных очень много классов. Исходя из этих данных выбран размер в 30 процентов, как сбалансированный вариант разбиения, подходящий под наши данные.

**Создание функции разбиения датасета на тренировочную и валидационную выборки.**

In [ ]:
# Инициализация функции разбиения датасета на тренировочную и валидационную выборки
def split_dataset(images_dir, labels_dir, output_dir, train_ratio=0.7):
    # Пути для сохранения разделенных данных
    train_images_dir = os.path.join(output_dir, "train/images")
    train_labels_dir = os.path.join(output_dir, "train/labels")
    val_images_dir = os.path.join(output_dir, "val/images")
    val_labels_dir = os.path.join(output_dir, "val/labels")
    
    # Создание папок, если их нет
    for folder in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        os.makedirs(folder, exist_ok=True)
    
    # Получаем список изображений и перемешиваем
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(image_files)
    
    # Определяем границу разделения 70/30
    split_idx = int(train_ratio * len(image_files))
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]
    
    # Функция для копирования изображений и соответствующих аннотаций
    def move_files(file_list, dst_images, dst_labels):
        for file in file_list:
            # Перемещение изображения
            shutil.copy2(os.path.join(images_dir, file), os.path.join(dst_images, file))
            
            # Перемещение аннотации (если есть)
            label_file = os.path.splitext(file)[0] + ".txt"
            src_label_path = os.path.join(labels_dir, label_file)
            dst_label_path = os.path.join(dst_labels, label_file)
            
            if os.path.exists(src_label_path):
                shutil.copy2(src_label_path, dst_label_path)
    
    # Копируем файлы
    move_files(train_files, train_images_dir, train_labels_dir)
    move_files(val_files, val_images_dir, val_labels_dir)
    
    print(f"Разбиение завершено: {len(train_files)} тренировочных, {len(val_files)} валидационных файлов.")

Разбиение завершено: 6727 тренировочных, 2883 валидационных файлов.


**Разделение датасета на выборки.**

In [ ]:
# Задать пути к данным
images_path = "images2"
labels_path = "labels_bbox"
output_path = "data"

# Запустить разбиение
split_dataset(images_path, labels_path, output_path)

**Итог:**

Датасет разбит на подвыборки.

## Выбор архитектуры

Необходимо разработать два подхода:

1. Без дообучения модели. 
2. С дообучением модели.

Выберем архитектуры для обоих подходов.

**Без дообучения модели**

Сиамская сеть идеально подходит для задачи сравнения изображений, так как она обучена выявлять сходства или различия между двумя изображениями. В вашем случае, когда новые фотографии сотрудников добавляются в базу данных, задача сводится к тому, чтобы сравнить новую фотографию с уже имеющимися в базе. Сиамская сеть решает эту задачу, эффективно используя два идентичных нейронных пути для извлечения признаков из изображений и последующего сравнения этих признаков с использованием метрики сходства (например, косинусного расстояния или евклидова расстояния). Такой подход позволяет распознавать новые фотографии, не требуется переподготовка всей модели, и это удобно для системы, где база данных постоянно обновляется новыми изображениями.

**С дообучением модели**

YOLO (You Only Look Once) — это одна из самых популярных моделей для детекции объектов, которая способна эффективно обнаруживать и классифицировать объекты на изображении в реальном времени. Для задачи с дообучением модель YOLO является оптимальной, так как она может быть адаптирована для классификации лиц или других объектов с помощью fine-tuning. В процессе дообучения модель будет адаптироваться к новым данным, улучшая свою точность на основе свежих изображений сотрудников. Благодаря возможности дообучения YOLO на новых данных без значительных изменений в структуре модели, она идеально подходит для задачи, где необходимо интегрировать новые фотографии в обучающий набор и обучать модель на расширенной базе данных. Это также дает возможность модели улучшать свои результаты с течением времени, не требуя полной переобучения с нуля.

Таким образом, выбранные архитектуры идеально соответствуют требованиям задачи: сиамская сеть предоставляет быстрый и эффективный способ сравнения новых изображений с базой данных без необходимости дообучения, а YOLO подходит для динамичного процесса улучшения модели через добавление новых данных и дообучение.

### Разработка YOLO модели

**Создание data.yaml**

Необходимо создать файл с информацией о датасете, для работы с изображениями с помощью Yolo.

In [ ]:
# инициализация пути
file_path = 'labels.txt'

# Словарь для хранения классов
labels = []

# Чтение файла и извлечение классов
with open(file_path, 'r') as f:
    for line in f:
        image, label = line.split()
        labels.append(int(label))

# Подсчет количества каждого класса
label_counts = Counter(labels)

# Извлечение меток и их частот
labels_list = list(label_counts.keys())
counts_list = list(label_counts.values())

In [ ]:
labels_list.append(52)

In [ ]:
labels_list = sorted(set(labels_list))  # Упорядочиваем классы
mapping = {old_id: new_id for new_id, old_id in enumerate(labels_list)} # Размечаем как словарь для создания yolo датасета

# вывод словаря
mapping

In [7]:
labels_dir = "D:\\data science\\B\\data\\train\\labels"

for label_file in os.listdir(labels_dir):
    file_path = os.path.join(labels_dir, label_file)
    with open(file_path, "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.split()
        class_id = int(parts[0])
        if class_id in mapping:
            parts[0] = str(mapping[class_id])  # Заменяем старый ID на новый
            new_lines.append(" ".join(parts))

    with open(file_path, "w") as f:
        f.writelines("\n".join(new_lines))

In [8]:
labels_dir = "D:\\data science\\B\\data\\val\\labels"

for label_file in os.listdir(labels_dir):
    file_path = os.path.join(labels_dir, label_file)
    with open(file_path, "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.split()
        class_id = int(parts[0])
        if class_id in mapping:
            parts[0] = str(mapping[class_id])  # Заменяем старый ID на новый
            new_lines.append(" ".join(parts))

    with open(file_path, "w") as f:
        f.writelines("\n".join(new_lines))

In [9]:
data = {
    'train': 'data/train/images',
    'val': 'data/val/images',
    'nc': len(labels_list),  # Должно быть 3483
    'names': [str(x) for x in labels_list],  # Список имен классов (в порядке от 0 до 3482)
    'train_labels': 'data/train/labels',
    'val_labels': 'data/val/labels'
}

# Сохранение в YAML
import yaml
with open('yolo_config_fixed.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False, allow_unicode=True)

print("✅ YAML обновлен!")

✅ YAML обновлен!


In [10]:
# import yaml

# # Пример данных для конфигурации YOLO
# data = {
#     'train': 'data/train/images',  # путь к обучающим изображениям
#     'val': 'data/val/images',  # путь к изображениям для валидации
#     'nc': len(labels_list),  # количество классов
#     'names': labels_list,  # имена классов
#     'train_labels': 'data/train/labels',  # путь к меткам обучающих данных
#     'val_labels': 'data/val/labels'  # путь к меткам валидационных данных
# }

# # Сохранение данных в YAML файл
# with open('yolo_config2test.yaml', 'w') as file:
#     yaml.dump(data, file, default_flow_style=True, allow_unicode=True)

# print("YAML файл успешно создан!")

In [47]:
def train(model_name, data_yaml):
    model = YOLO(model_name)
    training_results = model.train(
        data=data_yaml,
        epochs=5, # число эпох для обучения
        imgsz=240, # размер изображения для обучения
        batch=16, # размер батча для обучения
        device=0, # номер девайса для обучения
        single_cls=False # для обучения с учетом классов на основании data.yaml
    )

In [48]:
train("D:\\data science\\B\\images_v2\\yolov8m.pt", "D:\\data science\\B\\yolo_config_fixed.yaml")

New https://pypi.org/project/ultralytics/8.3.75 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.12.7 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine\trainer: task=detect, mode=train, model=D:\data science\B\images_v2\yolov8m.pt, data=D:\data science\B\yolo_config_fixed.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=240, save=True, save_period=-1, val_period=1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_

train: Scanning D:\data science\B\data\train\labels.cache... 9356 images, 2282 backgrounds, 5 corrupt: 100%|██████████| 9357/9357 [00:00<?, ?it/s]

train: WARNING ⚠️ D:\data science\B\data\train\images\frame_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        529         178         133         133]
train: WARNING ⚠️ D:\data science\B\data\train\images\frame_11.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        296         114         158         158]
train: WARNING ⚠️ D:\data science\B\data\train\images\frame_12.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        339         120         158         158]
train: WARNING ⚠️ D:\data science\B\data\train\images\frame_14.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        429         141         139         139]
train: WARNING ⚠️ D:\data science\B\data\train\images\frame_16.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        286         132         149         149]
albumentations: Blur(p=0.01, blur_limit=(

KeyboardInterrupt: 

In [25]:
model = YOLO(r'D:\data science\B\runs\detect\train\weights\best.pt')

In [44]:

result = model.predict(r"D:\data science\B\data\val\images\000008.jpg")
img_masks = result[0].plot(boxes=True, labels=False) 


image 1/1 D:\data science\B\data\val\images\000008.jpg: 256x192 (no detections), 55.4ms
Speed: 1.0ms preprocess, 55.4ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 192)


In [45]:
sv.plot_image(img_masks)

<Figure size 1200x1200 with 1 Axes>

In [46]:
plt.imsave("output.jpg", img_masks)

### Разработка сиамской модели

In [51]:
IMAGE_FOLDER = r"D:\data science\B\images2"
LABELS_FOLDER = r"D:\data science\B\labels_bbox"
BATCH_SIZE = 64
EPOCHS = 1
LR = 0.001

In [52]:
def load_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image)  # Убираем unsqueeze(0), чтобы было 3D

class SiameseDataset(Dataset):
    def __init__(self, image_folder, labels_folder):
        self.image_folder = image_folder
        self.labels_folder = labels_folder
        self.image_files = os.listdir(image_folder)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img1_name = self.image_files[idx]
        img2_name = np.random.choice(self.image_files)

        img1 = self.transform(Image.open(os.path.join(self.image_folder, img1_name)).convert('RGB'))
        img2 = self.transform(Image.open(os.path.join(self.image_folder, img2_name)).convert('RGB'))

        with open(os.path.join(self.labels_folder, img1_name.replace('.jpg', '.txt')), 'r') as f:
            label1 = int(f.readline().split()[0])
        
        with open(os.path.join(self.labels_folder, img2_name.replace('.jpg', '.txt')), 'r') as f:
            label2 = int(f.readline().split()[0])

        same_class = torch.tensor(1.0 if label1 == label2 else 0.0)
        return img1, img2, same_class


In [53]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(512, 256)
        self.fc = nn.Sequential(
            nn.Linear(256 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img1, img2):
        out1 = self.cnn(img1)
        out2 = self.cnn(img2)
        combined = torch.cat((out1, out2), dim=1)
        return self.fc(combined)

In [54]:
siamese_dataset = SiameseDataset(IMAGE_FOLDER, LABELS_FOLDER)
dataloader = DataLoader(siamese_dataset, batch_size=BATCH_SIZE, shuffle=True)
model = SiameseNetwork().cuda()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for img1, img2, labels in tqdm(dataloader):
        img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(img1, img2).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss/len(dataloader):.4f}")

torch.save(model.state_dict(), "siamese_model.pth")

  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 1/1, Loss: 0.0143


In [57]:
def load_image2(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0) # Добавляем batch dimension

In [58]:
IMAGE_FOLDER2 = r"D:\data science\B\images2"
LABELS_FOLDER2 = r"D:\data science\B\labels_bbox"

In [59]:
def predict_class(test_img_path, model_path="siamese_model.pth"):
    model = SiameseNetwork().cuda()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    test_img = load_image2(test_img_path).cuda()
    max_similarity = 0
    best_class = None
    
    for img_name in tqdm(os.listdir(IMAGE_FOLDER2)):
        img_path = os.path.join(IMAGE_FOLDER2, img_name)
        label_path = os.path.join(LABELS_FOLDER2, img_name.replace('.jpg', '.txt'))
        
        ref_img = load_image2(img_path).cuda()
        with open(label_path, 'r') as f:
            first_line = f.readline().strip()
            label = int(first_line.split()[0]) if first_line else 0
        
        with torch.no_grad():
            similarity = model(test_img, ref_img).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_class = label
    
    return best_class

# Пример предсказания
img_test =r"D:\data science\world_skils\osn\images_v2\images\000008.jpg"
predicted_class = predict_class(img_test)
print(f"Изображение принадлежит к классу: {predicted_class}")

  0%|          | 0/9610 [00:00<?, ?it/s]

Изображение принадлежит к классу: 407


In [61]:
import matplotlib.pyplot as plt

def validate_model(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    losses = []
    accuracies = []
    
    with torch.no_grad():
        for img1, img2, labels in tqdm(dataloader):
            img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()
            outputs = model(img1, img2).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
        avg_loss = val_loss / len(dataloader)
        accuracy = correct / total
        losses.append(avg_loss)
        accuracies.append(accuracy)
    
    return losses, accuracies

# Вызов функции валидации и построение графика
val_losses, val_accuracies = validate_model(model, dataloader, criterion)

def plot_metrics(val_losses, val_accuracies):
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    
    ax1.plot(val_losses, 'r-', label='Validation Loss')
    ax2.plot(val_accuracies, 'b-', label='Validation Accuracy')
    
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='r')
    ax2.set_ylabel('Accuracy', color='b')
    
    plt.title('Validation Metrics')
    fig.legend(loc='upper right')
    plt.show()

plot_metrics(val_losses, val_accuracies)

  0%|          | 0/151 [00:00<?, ?it/s]

<Figure size 640x480 with 2 Axes>